In [1]:
# json file : javascript object notation
# primarly used for transmitting data between web app and a server
# lightweight, text based, human-readable

import json
filename = "data/frames/frames_ex.json"
with open(filename) as json_file:
    exemple = json.load(json_file)
print(exemple[0]['turns'][0]['text'])
exemple[0]['turns'][0]['labels']

I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700.


{'acts': [{'args': [{'val': 'book', 'key': 'intent'}], 'name': 'inform'},
  {'args': [{'val': 'Atlantis', 'key': 'dst_city'},
    {'val': 'Caprica', 'key': 'or_city'},
    {'val': 'Saturday, August 13, 2016', 'key': 'str_date'},
    {'val': '8', 'key': 'n_adults'},
    {'val': '1700', 'key': 'budget'}],
   'name': 'inform'}],
 'acts_without_refs': [{'args': [{'val': 'book', 'key': 'intent'}],
   'name': 'inform'},
  {'args': [{'val': 'Atlantis', 'key': 'dst_city'},
    {'val': 'Caprica', 'key': 'or_city'},
    {'val': 'Saturday, August 13, 2016', 'key': 'str_date'},
    {'val': '8', 'key': 'n_adults'},
    {'val': '1700', 'key': 'budget'}],
   'name': 'inform'}],
 'active_frame': 1,
 'frames': [{'info': {'intent': [{'val': 'book', 'negated': False}],
    'budget': [{'val': '1700.0', 'negated': False}],
    'dst_city': [{'val': 'Atlantis', 'negated': False}],
    'or_city': [{'val': 'Caprica', 'negated': False}],
    'str_date': [{'val': 'august 13', 'negated': False}],
    'n_adults': 

on a besoin d'extraire les informations suivantes:
* Ville de départ
* Ville de destination
* Date aller souhaitée du vol
* Date retour souhaitée du vol
* Budget maximum pour le prix total des billets


In [2]:
import json
filename = "data/frames/frames.json"

turn_processed = 0
with open(filename) as json_file:
    # json.load() prend un file et retourne un json object
    # le json object contient des pairs key/value
    # key (strings) : value ,
    data = json.load(json_file)

    # d'aprés le premier bloc du json, l'info qui nous interesse
    # se trouve dans turns

    # recherche par user_id/ 1ere clé du json
    list_example = []
    for a, bloc_user in enumerate(data):
        # if a == 15:
        # break
        for b, turn in enumerate(bloc_user['turns']):
            # intitalisation
            text = turn['text']
            # print(text)
            intent_found = False
            intention = "none"

            # un seul 'labels' par turn
            for c, act in enumerate(turn['labels']['acts']):
                # initialisation d'une liste d'entités pour un texte
                entities = []

                for d, arg in enumerate(act['args']):
                    #print(b, " turn ", c, " act ", d, " arg :", arg)
                    # entity dict avec 3 clés: entity, beg, end
                    entity = dict()
                    # nom de l'entité
                    if arg['key'] == 'dst_city':
                        entity['entityName'] = 'dst_city'
                    elif arg['key'] == 'or_city':
                        entity['entityName'] = 'or_city'
                    elif arg['key'] == "budget":
                        entity["entityName"] = "budget"
                    elif arg['key'] == "str_date":
                        entity["entityName"] = "str_date"
                    elif arg['key'] == "end_date":
                        entity["entityName"] = "end_date"

                        # si la key est une entité qu'on recherche et qu il y a une valeur
                    if arg['key'] in ["dst_city", "or_city", "budget", "str_date", "end_date"]\
                            and 'val' in arg.keys():
                        # position dans le text des entités : -1 si on ne trouve pas
                        entity['startCharIndex'] = text.find(arg['val'])
                        entity['endCharIndex'] = entity['startCharIndex'] + \
                            len(arg['val'])-1

                        # on ajoute l'entité que si on trouve sa position
                        if entity['startCharIndex'] != -1:
                            entities.append(entity)

                    if arg['key'] == 'intent':
                        intent = arg['val']
                        intent_found = True
            # on ajoute l'exemple que si il a une intention trouvée
            if (intent_found and len(entities) >= 1):  # len(entities) > 0:  #
                example = dict()
                example['text'] = text
                example['entityLabels'] = entities
                if intent_found:
                    example['intentName'] = intent
                else:
                    example['intentName'] = "none"
                list_example.append(example)
        if len(list_example) > 30:
            break

In [3]:
list_example

[{'text': "I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700.",
  'entityLabels': [{'entityName': 'dst_city',
    'startCharIndex': 27,
    'endCharIndex': 34},
   {'entityName': 'or_city', 'startCharIndex': 41, 'endCharIndex': 47},
   {'entityName': 'str_date', 'startCharIndex': 52, 'endCharIndex': 76},
   {'entityName': 'budget', 'startCharIndex': 117, 'endCharIndex': 120}],
  'intentName': 'book'},
 {'text': "I'm looking for a trip to Gotham City leaving from Kakariko Village on Saturday, August 13, 2016. 3 adults for no more than $2400 USD.",
  'entityLabels': [{'entityName': 'dst_city',
    'startCharIndex': 26,
    'endCharIndex': 36},
   {'entityName': 'or_city', 'startCharIndex': 51, 'endCharIndex': 66},
   {'entityName': 'str_date', 'startCharIndex': 71, 'endCharIndex': 95},
   {'entityName': 'budget', 'startCharIndex': 124, 'endCharIndex': 132}],
  'intentName': 'book'},
 {'text': 'Hello, I would like to 

### creation, initalisation de l'app et du client authoring

In [4]:
from azure.cognitiveservices.language.luis.authoring import LUISAuthoringClient
from azure.cognitiveservices.language.luis.authoring.models import ApplicationCreateObject
from azure.cognitiveservices.language.luis.runtime import LUISRuntimeClient
from msrest.authentication import CognitiveServicesCredentials
from functools import reduce

import json
import time
import uuid

authoringKey = '2e9b1d71c34c40e09607079d58f2140e'
authoringEndpoint = 'https://p10luisinst-authoring.cognitiveservices.azure.com//'
#predictionKey = 'f992f91ca10a40a787f153930f4da0d5'
#predictionEndpoint = 'https://nab-luis-r.cognitiveservices.azure.com/'

# We use a UUID to avoid name collisions.
# nom de l'app
appName = "P10 mvp book"  # + str(uuid.uuid4())
versionId = "0.1"
# nom de l'intention
intentName = "FlyMeIntent"
culture = "en-us"

In [5]:
# creation d'un objet LuisAuthoringClient
client = LUISAuthoringClient(
    authoringEndpoint, CognitiveServicesCredentials(authoringKey))

# L'app LUIS stocke le modele de traitement de language
# qui contient intentions, entités et exemples d'enoncés (utterances)

# definition des bases de l'app
appDefinition = ApplicationCreateObject(
    name=appName, initial_version_id=versionId, culture=culture)

# creation de l'app et recup de l'id en string
app_id = client.apps.add(appDefinition)

# get app id - necessary for all other changes
print("Created LUIS app with ID {}".format(app_id))

Created LUIS app with ID 742890f8-77f9-4b15-8f45-257554e9e900


### creation du schema

In [6]:
# ajout d'un modèle d'intention:
intention = "book"
client.model.add_intent(app_id, versionId, intention)

'2f3e3e8e-503b-40d8-85e6-1883654fc365'

les differents types d'entités:  
* https://docs.microsoft.com/fr-fr/azure/cognitive-services/luis/concepts/entities
* **list entity**:
    * https://docs.microsoft.com/fr-fr/azure/cognitive-services/luis/reference-entity-list?tabs=V2
    * ensemble connu fixe de mots associés et de synonymes peu variables
    * pas issu du ML
    * exemple: liste "navy","royal","baby" pour blue
* **regex entity**
    * ideal pour du texte structuré
    * exemple : vol [A-Z]{2} [0-9]{4} -> vol AS 1234 par ex
* **prebuilt entity**
    * https://docs.microsoft.com/fr-fr/azure/cognitive-services/luis/luis-reference-prebuilt-entities
    * entités predefinies : date, heure, nombre ....variant selon la culture d'ou l'importance e la specifier
* **pattern.any entity**
    * https://docs.microsoft.com/fr-fr/azure/cognitive-services/luis/reference-entity-pattern-any?tabs=V2
    * entité qui reserve un espace variable dans un énoncé pour marquer le debut et la fin d'uen entité
    * convient aux phrases à structure lexicale fixe: 
        * exemple: puis je avoir un {entité plat} s'il vous plait ?
    * à marquer dans les exemples de modèles Pattern et non dans les exemples d'intention
* **ML entity**
    * https://docs.microsoft.com/fr-fr/azure/cognitive-services/luis/reference-entity-machine-learned-entity?tabs=V3
    * entity issu du machine learning
    * utilise le contexte pour extraire des entités en fonction d'exemples etiquetés

In [7]:
# ajout de modèles d'entités prebuilt
# ils serviront comme features prelabelisées à d'autres modèles d'entités

# https://docs.microsoft.com/en-us/azure/cognitive-services/luis/luis-reference-prebuilt-entities#english-american-entity-support
# https://docs.microsoft.com/en-us/azure/cognitive-services/luis/concepts/patterns-features

# attention donner les bons noms qui commencent en minuscule
# on aura une erreur si mauvais nom ou si deja added
client.model.add_prebuilt(
    app_id, versionId, prebuilt_extractor_names=["money", "geographyV2", "datetimeV2"])

In [8]:
# ajout d'un modèle d'entité pour le budget du voyage
Budget_Id = client.model.add_entity(
    app_id, versionId, name="budget")

# ajout d'un currency entity prebuilt model comme feature de l'entité budget
client.features.add_entity_feature(
    app_id, versionId, Budget_Id, {"model_name": "money", "is_required": False})

In [9]:
# premier jet peut etre trop specifique: il faudra certainement raffiner en utilisant tous les tricks

# ajout d'un modèle d'entité pour le lieu de depart
From_Id = client.model.add_entity(
    app_id, versionId, name="or_city")

# ajout d'un GeographyV2 entity prebuilt model comme feature de l'entité from
client.features.add_entity_feature(
    app_id, versionId, From_Id, {"model_name": "geographyV2", "is_required": False})

# ajout d'un modèle d'entité pour le lieu d'arrivée
To_Id = client.model.add_entity(
    app_id, versionId, name="dst_city")

# ajout d'un GeographyV2 entity prebuilt model comme feature de l'entité to
client.features.add_entity_feature(
    app_id, versionId, To_Id, {"model_name": "geographyV2", "is_required": False})

In [10]:
# premier jet peut etre trop specifique: il faudra certainement raffiner en utilisant tous les tricks

# ajout d'un modèle d'entité pour la date de depart
DateIn_Id = client.model.add_entity(
    app_id, versionId, name="str_date")

# ajout d'un datetimeV2 entity prebuilt model comme feature de l'entité from
client.features.add_entity_feature(
    app_id, versionId, DateIn_Id, {"model_name": "datetimeV2", "is_required": False})

# ajout d'un modèle d'entité pour la date de retour
DateOut_Id = client.model.add_entity(
    app_id, versionId, name="end_date")

# ajout d'un datetimeV2 entity prebuilt model comme feature de l'entité to
client.features.add_entity_feature(
    app_id, versionId, DateOut_Id, {"model_name": "datetimeV2", "is_required": False})

In [11]:
# ajout des enoncés labelisés
# add pour un seul exemple/batch pour un lot d'exemples
client.examples.batch(app_id, versionId, list_example, {
    "enableNestedChildren": True})

In [12]:
# entrainement du modèle LUIS
# un modèle tres petit sera trés rapide à entrainer

# lancement de l'entrainement
client.train.train_version(app_id, versionId)
waiting = True
while waiting:
    # recuperation du status pour savoir si l'netrainement a reussi (un par modèle)
    info = client.train.get_status(app_id, versionId)

    # get_status gets the training status of all models (intents and entities) for the specified LUIS app.
    # Loop through them and make sure all are done.
    # map : iterateur qui va renvoyer une liste de TRUE si on a du queued ou du in progress
    # any renvoie true si au moins un element dun iterable est true
    # waiting passe à false et on sort du while quand le train est fini
    waiting = any(map(lambda x: 'Queued' ==
                  x.details.status or 'InProgress' == x.details.status, info))
    if waiting:
        print("Waiting 10 seconds for training to complete...")
        time.sleep(10)
    else:
        print("trained")
        waiting = False

Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
trained


In [13]:
for model in info:
    print("model id:", model.model_id, "- status:", model.details.status,
          "- nb exemples:", model.details.example_count,
          "- training date:", model.details.training_date_time,
          "- failure reason", model.details.failure_reason)

model id: 2f3e3e8e-503b-40d8-85e6-1883654fc365 - status: Success - nb exemples: 31 - training date: 2022-07-05 12:30:18+00:00 - failure reason None
model id: 5bc2ebeb-7e73-4625-966b-fa7742fac86a - status: Success - nb exemples: 31 - training date: 2022-07-05 12:30:18+00:00 - failure reason None
model id: a9f35975-f3c4-44e4-9def-dbdc58c68f88 - status: Success - nb exemples: 31 - training date: 2022-07-05 12:30:18+00:00 - failure reason None
model id: ff59aba1-df4c-4ff5-a7c2-7410c9d0428a - status: Success - nb exemples: 31 - training date: 2022-07-05 12:30:19+00:00 - failure reason None
model id: 0b35c015-0719-4385-b881-455a75eb339c - status: Success - nb exemples: 31 - training date: 2022-07-05 12:30:21+00:00 - failure reason None
model id: 87704c11-57ff-4f8a-bd79-d10f68f3c174 - status: Success - nb exemples: 31 - training date: 2022-07-05 12:30:21+00:00 - failure reason None
model id: ba2e0735-380d-48f8-85ed-d9b218775acd - status: Success - nb exemples: 31 - training date: 2022-07-05 1

In [14]:
# publication du modele entrainé
# vaut mieux mettre l'app en publi pour la query de n'importe quel endpoint

# Note: For production scenarios, you should instead assign the app to your own LUIS prediction endpoint. See:
# https://docs.microsoft.com/en-gb/azure/cognitive-services/luis/luis-how-to-azure-subscription#assign-a-resource-to-an-app

client.apps.update_settings(app_id, is_public=True)
responseEndpointInfo = client.apps.publish(app_id, versionId, is_staging=False)

In [15]:
# https://docs.microsoft.com/en-us/python/api/azure-cognitiveservices-language-luis/azure.cognitiveservices.language.luis.authoring.models.productionorstagingendpointinfo?view=azure-python
print("version_id:", responseEndpointInfo.version_id)
print("is_staging:", responseEndpointInfo.is_staging)
print("endpoint_url:", responseEndpointInfo.endpoint_url)
print("region:", responseEndpointInfo.region)
print("assigned_endpoint_key:", responseEndpointInfo.assigned_endpoint_key)
print("endpoint_region:", responseEndpointInfo.endpoint_region)
print("failed_regions:", responseEndpointInfo.failed_regions)
print("published_date_time:", responseEndpointInfo.published_date_time)

version_id: 0.1
is_staging: False
endpoint_url: https://westeurope.api.cognitive.microsoft.com/luis/v2.0/apps/742890f8-77f9-4b15-8f45-257554e9e900
region: westeurope, northeurope, uksouth, ukwest, francecentral, francesouth, norwayeast, norwaywest
assigned_endpoint_key: None
endpoint_region: westeurope
failed_regions: 
published_date_time: 2022-07-05T12:36:56Z


In [16]:
app_id

'742890f8-77f9-4b15-8f45-257554e9e900'

In [17]:
# authentifier le client du runtime de prediction (on peut aussi utiliser l'authoring)
# clientRuntime = LUISRuntimeClient(
# endpoint=predictionEndpoint, credentials=CognitiveServicesCredentials(predictionKey))
clientRuntime = LUISRuntimeClient(
    endpoint=authoringEndpoint, credentials=CognitiveServicesCredentials(authoringKey))
app_id = "742890f8-77f9-4b15-8f45-257554e9e900"

In [18]:
# obtenir une prediction du runtime
# "Production" = slot name
predictionRequest = {
    "query": "Looking to go from San Francisco to MArseille. Book me for September 18 to 22. Let me know if its more than 2800 because thats all I can afford"}
# "query": "I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700."}

# get_slot_prediction : slot_name:"production"
# https://docs.microsoft.com/en-us/python/api/azure-cognitiveservices-language-luis/azure.cognitiveservices.language.luis.runtime.operations.predictionoperations?view=azure-python#get-slot-prediction-app-id--slot-name--prediction-request--verbose-none--show-all-intents-none--log-none--custom-headers-none--raw-false----operation-config-"
predictionResponse = clientRuntime.prediction.get_slot_prediction(
    app_id, "Production", predictionRequest, verbose=True, show_all_intents=True)

# predictionResponse = clientRuntime.prediction.get_version_prediction(
#   app_id,versionId,predictionRequest, verbose=True, show_all_intents=True)

print("top scoring intent: {}".format(predictionResponse.prediction.top_intent))
print("Sentiment: {}".format(predictionResponse.prediction.sentiment))

top scoring intent: book
Sentiment: None


In [19]:
for intent in predictionResponse.prediction.intents:
    # print("\t{}".format(json.dumps(intent)))
    print("Intent : {} , Score : {}".format(
        intent, predictionResponse.prediction.intents[intent].score))

Intent : book , Score : 0.99933076
Intent : None , Score : 0.0027660364


In [20]:
for entity in predictionResponse.prediction.entities:
    print("Entity : {} , Value : {}".format(
        entity, predictionResponse.prediction.entities[entity]))

Entity : or_city , Value : ['San Francisco']
Entity : geographyV2 , Value : [{'value': 'San Francisco', 'type': 'city'}]
Entity : dst_city , Value : ['MArseille']
Entity : str_date , Value : ['September 18']
Entity : datetimeV2 , Value : [{'type': 'daterange', 'values': [{'timex': '(XXXX-09-18,XXXX-09-22,P4D)', 'resolution': [{'start': '2021-09-18', 'end': '2021-09-22'}, {'start': '2022-09-18', 'end': '2022-09-22'}]}]}]
Entity : $instance , Value : {'or_city': [{'type': 'or_city', 'text': 'San Francisco', 'startIndex': 19, 'length': 13, 'score': 0.9526819, 'modelTypeId': 1, 'modelType': 'Entity Extractor', 'recognitionSources': ['model']}], 'geographyV2': [{'type': 'builtin.geographyV2.city', 'text': 'San Francisco', 'startIndex': 19, 'length': 13, 'modelTypeId': 2, 'modelType': 'Prebuilt Entity Extractor', 'recognitionSources': ['model']}], 'dst_city': [{'type': 'dst_city', 'text': 'MArseille', 'startIndex': 36, 'length': 9, 'score': 0.94613343, 'modelTypeId': 1, 'modelType': 'Entity 

In [21]:
for instance in predictionResponse.prediction.entities["$instance"]:
    print("Entity_name : {} , Value : {}".format(
        instance, predictionResponse.prediction.entities["$instance"][instance]))

Entity_name : or_city , Value : [{'type': 'or_city', 'text': 'San Francisco', 'startIndex': 19, 'length': 13, 'score': 0.9526819, 'modelTypeId': 1, 'modelType': 'Entity Extractor', 'recognitionSources': ['model']}]
Entity_name : geographyV2 , Value : [{'type': 'builtin.geographyV2.city', 'text': 'San Francisco', 'startIndex': 19, 'length': 13, 'modelTypeId': 2, 'modelType': 'Prebuilt Entity Extractor', 'recognitionSources': ['model']}]
Entity_name : dst_city , Value : [{'type': 'dst_city', 'text': 'MArseille', 'startIndex': 36, 'length': 9, 'score': 0.94613343, 'modelTypeId': 1, 'modelType': 'Entity Extractor', 'recognitionSources': ['model']}]
Entity_name : str_date , Value : [{'type': 'str_date', 'text': 'September 18', 'startIndex': 59, 'length': 12, 'score': 0.5378463, 'modelTypeId': 1, 'modelType': 'Entity Extractor', 'recognitionSources': ['model']}]
Entity_name : datetimeV2 , Value : [{'type': 'builtin.datetimeV2.daterange', 'text': 'September 18 to 22', 'startIndex': 59, 'lengt

https://docs.microsoft.com/en-us/azure/cognitive-services/luis/developer-reference-resource  
https://docs.microsoft.com/en-us/azure/cognitive-services/LUIS/luis-how-to-batch-test?tabs=rest

### test de l'API

In [39]:
# test creation via l'api

# https://westus.dev.cognitive.microsoft.com/docs/services/luis-programmatic-apis-v3-0-preview/operations/5890b47c39e2bb052c5b9c2f

# on a un endpoint en V2 plus haut
# donc on test avec v2

# LUIS Programmatic APIs v3.0-preview
import http.client  # implement the client side of the HTTP and HTTPS protocols
# functions and classes which help in opening URLs (mostly HTTP) in a complex world
import urllib.request
# standard interface to break Uniform Resource Locator (URL) strings up in components
import urllib.parse
import urllib.error  # defines the exception classes for exceptions raised by urllib.request
import base64  # provides functions for encoding binary data to printable ASCII characters and decoding such encodings back to binary data

######################
headers = {
    # Request headers
    'Content-Type': 'application/json',
    # key du nab-luis
    'Ocp-Apim-Subscription-Key': '46739733dafd4a63bba53c8d40980f5e'
}

params = urllib.parse.urlencode({
})

body = {
    "name": "MyFirstDummyApp",
    "description": "This is my first dummy application",
    "culture": "en-us",
    "tokenizerVersion": "1.0.0",
    "usageScenario": "IoT",
    "domain": "Comics",
    "initialVersionId": "1.0"}

# https://nab-luis.cognitiveservices.azure.com/
try:
    conn = http.client.HTTPSConnection(
        'westeurope.api.cognitive.microsoft.com')
    conn.request("POST", "/luis/api/v2.0/apps/?%s" %
                 params, str(body), headers)
    response = conn.getresponse()
    data = response.read()
    print(data)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

b'"aceffde9-9b64-4ac0-9f05-93ee82b7589c"'


In [44]:
headers = {
    # Request headers
    'Ocp-Apim-Subscription-Key': '46739733dafd4a63bba53c8d40980f5e'
}
body = {}
appId = "aceffde9-9b64-4ac0-9f05-93ee82b7589c"

params = urllib.parse.urlencode({
    # Request parameters
    'force': 'false',
})

try:
    conn = http.client.HTTPSConnection(
        'westeurope.api.cognitive.microsoft.com')
    conn.request(
        "DELETE", "/luis/api/v2.0/apps/"+appId+"?%s" % params, "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    print(data)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

####################################

b'{"code":"Success","message":"Operation Successful"}'


In [68]:
# pour envoyer des web post requests
# The post() method sends a POST request to the specified url.
# The post() method is used when you want to send some data to the server
# syntaxe: requests.post(url, data={key: value}, json={key: value}, args)
import json
import requests
# <YOUR-PREDICTION-ENDPOINT>/luis/v3.0-preview/apps/<YOUR-APP-ID>/versions/<YOUR-APP-VERSION-ID>/evaluations
#url = 'https://luis2nab.cognitiveservices.azure.com/luis/v3.0-preview/apps/3e41b012-8072-4b01-8280-9cafba54ba91/versions/0.1/evaluations'
url = 'https://westeurope.api.cognitive.microsoft.com/luis/v3.0-preview/apps/851fc277-528d-42fe-8e2a-4d7fb07c27de/versions/0.1/evaluations'
myobj = {
    "LabeledTestSetUtterances": [{"text": "I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700.",
                                  "intent": "book",
                                  "entities": [{"entity": "to", "startPos": 27, "endPos": 34},
                                               {"entity": "from",
                                                "startPos": 41, "endPos": 47},
                                               {"entity": "datin", "startPos": 52,
                                                   "endCharIndex": 76},
                                               {"entity": "budget", "startPos": 117, "endPos": 120}]}]
}

In [69]:
#myfiles = {'file': open("my_json_file", 'rb')}

# use the 'headers' parameter to set the HTTP headers:
x = requests.post(url, json=myobj, headers={
                  'Ocp-Apim-Subscription-Key': '46739733dafd4a63bba53c8d40980f5e',
                  'Content-Type': 'application/json'})

In [70]:
x.text

'{"operationId":"6b89b6cd-f4fb-46fa-9976-20068ae42989_637900704000000000","status":"notstarted","createdDateTime":"06/06/2022 22:25:50","lastActionDateTime":"06/06/2022 22:25:50"}'

In [71]:
# get the status
# <YOUR-PREDICTION-ENDPOINT > /luis/v3.0-preview/apps/<YOUR-APP-ID > /versions/<YOUR-APP-VERSION-ID > /evaluations/<YOUR-OPERATION-ID > /status
url = 'https://westeurope.api.cognitive.microsoft.com/luis/v3.0-preview/apps/851fc277-528d-42fe-8e2a-4d7fb07c27de/versions/0.1/evaluations/6b89b6cd-f4fb-46fa-9976-20068ae42989_637900704000000000/status'
x = requests.get(url, headers={
    'Ocp-Apim-Subscription-Key': '46739733dafd4a63bba53c8d40980f5e'})

In [72]:
x.text

'{"errorDetails":"{\\"error\\":{\\"code\\":\\"NotFound\\",\\"message\\":\\"The application wasn\'t found. Please make sure that the application is published to this region.\\"}}","operationId":"6b89b6cd-f4fb-46fa-9976-20068ae42989_637900704000000000","status":"failed","createdDateTime":"06/06/2022 22:25:50","lastActionDateTime":"06/06/2022 22:25:50"}'

In [35]:
recognizer_result = {
    "text": "i want to travel from paris to london on june 10 2022 and return on june 13 2022 with a budget of 2000 $",
    "alteredText": {},
    "intents": {
        "book": {
            "score": 0.99797785
        }
    }}

intent = (
    sorted(
        recognizer_result.get("intents"),
        key=recognizer_result.get("intents").get,
        reverse=True,
    )[:1][0]
    if recognizer_result.get("intents")
    else None
)

In [36]:
intent

'book'

In [47]:
recognizer_result = {
    "text": "i want to travel from paris to london on june 10 2022 and return on june 13 2022 with a budget of 2000 $",
    "alteredText": {},
}

# le {} permet detre retourner en defaut si rien pour eviter uen erreur su rle prochain get
recognizer_result.get("intents", {}).get("dst_city", [])

[]

In [51]:
h = recognizer_result.get("date", [])
if h:
    print("wesh")

In [52]:
h

[]

In [2]:
recognizer_result = {
    "text": "i want to travel from paris to london on june 10 2022 and return on june 13 2022 with a budget of 2000 $",
    "alteredText": {},
    "intents": {
        "book": {
            "score": 0.99797785
        }
    },
    "entities": {
        "$instance": {
            "budget": [
                {
                    "startIndex": 98,
                    "endIndex": 104,
                    "text": "2000 $",
                    "type": "budget",
                    "score": 0.7724428
                }
            ],
            "or_city": [
                {
                    "startIndex": 22,
                    "endIndex": 27,
                    "text": "paris",
                    "type": "or_city",
                    "score": 0.84419185
                }
            ],
            "str_date": [
                {
                    "startIndex": 41,
                    "endIndex": 48,
                    "text": "june 10",
                    "type": "str_date",
                    "score": 0.34791893
                }
            ],
            "datetime": [
                {
                    "startIndex": 41,
                    "endIndex": 53,
                    "text": "june 10 2022",
                    "type": "builtin.datetimeV2.date"
                },
                {
                    "startIndex": 68,
                    "endIndex": 80,
                    "text": "june 13 2022",
                    "type": "builtin.datetimeV2.date"
                },
                {
                    "startIndex": 98,
                    "endIndex": 102,
                    "text": "2000",
                    "type": "builtin.datetimeV2.daterange"
                }
            ],
            "geographyV2_city": [
                {
                    "startIndex": 22,
                    "endIndex": 27,
                    "text": "paris",
                    "type": "builtin.geographyV2.city"
                },
                {
                    "startIndex": 31,
                    "endIndex": 37,
                    "text": "london",
                    "type": "builtin.geographyV2.city"
                }
            ],
            "money": [
                {
                    "startIndex": 98,
                    "endIndex": 104,
                    "text": "2000 $",
                    "type": "builtin.currency"
                }
            ]
        },


        "budget": [
            "2000 $"
        ],
        "or_city": [
            "paris"
        ],
        "str_date": [
            "june 10"
        ],
        "datetime": [
            {
                "type": "date",
                "timex": [
                    "2022-06-10"
                ]
            },
            {
                "type": "date",
                "timex": [
                    "2022-06-13"
                ]
            },
            {
                "type": "daterange",
                "timex": [
                    "2000"
                ]
            }
        ],
        "geographyV2_city": [
            "paris",
            "london"
        ],
        "money": [
            {
                "number": 2000,
                "units": "Dollar"
            }
        ]
    }
}

In [ ]:
datetime_entities = recognizer_result.entities.get("datetime", [])
        for i in datetime_entities:
        if datetime_entities[i]["startIndex"] == recognizer_result.entities.get("str_date", []).get("startIndex", {}):
        print("builtin.datetimeV2.date":
    result.departure_date=datetime_entities[0]['timex'][0]

In [2]:
"2122-06-10" >= "2022-06-10"

True